#Downloading required packages

In [1]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#Importing libraries

In [2]:
import tarfile
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import tensorflow as tf
from sklearn.metrics import precision_score

#Extracting the Dataset files

In [4]:
file_path = '/content/review_polarity.tar.gz'
extract_path = '.'

# Extract the tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

print("Extraction complete!")


Extraction complete!


#Preprocessing

In [5]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stop words and lemmatize
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words and word.isalpha()]

    return ' '.join(processed_tokens)

def load_data_from_folder(folder, label):
    data = []
    for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            processed_content = preprocess_text(content)
            data.append((processed_content, label))
    return data


neg_folder='/content/txt_sentoken/neg'
pos_folder='/content/txt_sentoken/pos'

neg_data = load_data_from_folder(neg_folder, 0)
pos_data = load_data_from_folder(pos_folder, 1)

# Combine data and create a DataFrame
all_data = neg_data + pos_data
df = pd.DataFrame(all_data, columns=['text', 'label'])

print(df.head())


                                                text  label
0  could disney film based meg cabot novel prince...      0
1  warning spoiler included review really make mu...      0
2  attempt florida film noir palmetto fails funda...      0
3  shagadellic groovy baby smashing adorn page ad...      0
4  starring william baldwin cindy crawford steven...      0


In [6]:
print(len(neg_data))
print(len(pos_data))

1000
1000


#**First Approach: using unigrams**


In [ ]:
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df['text'])
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###Logistic Regression

In [ ]:
clf = LogisticRegression()

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.80      0.81       199
           1       0.81      0.82      0.81       201

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400



###SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


# Initialize SVM classifier
clf = SVC(kernel='linear', random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.815
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       199
           1       0.82      0.82      0.82       201

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400



###Neural network

In [ ]:
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Initialize the Neural Network model
model = Sequential()
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.75))  # Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.75))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Compile the model with SGD + Momentum
sgd = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Convert the sparse matrix to a dense array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Train the model using the dense arrays
history = model.fit(X_train_dense, y_train, batch_size=32, epochs=80,
                    validation_data=(X_test_dense, y_test), verbose=1)

# Predict on the test set
y_pred_probs = model.predict(X_test_dense)
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.83
Precision: 0.8151658767772512
Recall: 0.8557213930348259
F1-score: 0.8349514563106796
Confusion Matrix:
 [[160  39]
 [ 29 172]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.80      0.82       199
           1       0.82      0.86      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



####After some fine-tuning:

In [ ]:
from keras.optimizers import SGD

# Initialize the Neural Network model
model = Sequential()
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.75))  # Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.75))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Compile the model with SGD + Momentum
sgd = SGD(learning_rate=0.01, momentum=0.8)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Convert the sparse matrix to a dense array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Train the model using the dense arrays
history = model.fit(X_train_dense, y_train, batch_size=32, epochs=80,
                    validation_data=(X_test_dense, y_test), verbose=1)

# Predict on the test set
y_pred_probs = model.predict(X_test_dense)
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               17209344  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 17340929 (66.15 MB)
Trainable params: 17340929 (66.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/80
5

####Applying K-folds cross validation:

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

def create_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.25))  # Dropout for regularization
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))

    sgd = SGD(learning_rate=0.01, momentum=0.8)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model


kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

# Perform cross-validation
for train_index, val_index in kf.split(X.toarray(), y):
    X_train_cv, X_val_cv = X[train_index], X[val_index]
    y_train_cv, y_val_cv = y[train_index], y[val_index]

    model = create_model()

    # Train the model
    model.fit(X_train_cv.toarray(), y_train_cv, batch_size=16, epochs=80, verbose=1)

    # Evaluate on validation set
    y_pred_probs = model.predict(X_val_cv.toarray())
    y_pred = (y_pred_probs > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_val_cv, y_pred)
    precision = precision_score(y_val_cv, y_pred)
    recall = recall_score(y_val_cv, y_pred)
    f1 = f1_score(y_val_cv, y_pred)
    conf_matrix = confusion_matrix(y_val_cv, y_pred)

    # Append scores to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Compute average scores across folds
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)

print("Average Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1)


Epoch 1/80
100/100 [==============================] - 2s 6ms/step - loss: 0.6931 - accuracy: 0.5094
Epoch 2/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6924 - accuracy: 0.5250
Epoch 3/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6908 - accuracy: 0.5394
Epoch 4/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6889 - accuracy: 0.5706
Epoch 5/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6860 - accuracy: 0.6012
Epoch 6/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6824 - accuracy: 0.6075
Epoch 7/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6760 - accuracy: 0.7450
Epoch 8/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6690 - accuracy: 0.7356
Epoch 9/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6573 - accuracy: 0.7631
Epoch 10/80
100/100 [==============================] - 1s 6ms/step - loss: 0.6416 - accuracy: 0.8087

#**Second Approach: N-grams**

##Using 5000 features

In [7]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

texts = df['text'].tolist()
labels = df['label']

# Initialize TfidfVectorizer for TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_features=5000, tokenizer=lambda x: x.split())
X_tfidf = vectorizer.fit_transform(texts)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Convert sparse matrix to dense array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
print(X_train.shape)

(1600, 5000)


###Logistic Regression

In [9]:
clf = LogisticRegression()

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8325
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       199
           1       0.85      0.82      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



###SVM

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


# Initialize SVM classifier
clf = SVC(kernel='linear', random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.84
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       199
           1       0.85      0.83      0.84       201

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



###Neural Network

In [11]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(X_train_dense.shape[1],), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_dense, y_train, epochs=80, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred_proba = model.predict(X_test_dense)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Display classification report
print('\nClassification Report:')
print(classification_report(y_test, y_pred))


Epoch 1/80
45/45 [==============================] - 2s 21ms/step - loss: 0.6718 - accuracy: 0.6826 - val_loss: 0.6375 - val_accuracy: 0.8250
Epoch 2/80
45/45 [==============================] - 1s 14ms/step - loss: 0.5390 - accuracy: 0.8896 - val_loss: 0.5328 - val_accuracy: 0.8313
Epoch 3/80
45/45 [==============================] - 1s 12ms/step - loss: 0.3711 - accuracy: 0.9257 - val_loss: 0.4413 - val_accuracy: 0.8313
Epoch 4/80
45/45 [==============================] - 0s 10ms/step - loss: 0.2456 - accuracy: 0.9583 - val_loss: 0.3965 - val_accuracy: 0.8250
Epoch 5/80
45/45 [==============================] - 0s 9ms/step - loss: 0.1684 - accuracy: 0.9792 - val_loss: 0.3729 - val_accuracy: 0.8438
Epoch 6/80
45/45 [==============================] - 0s 10ms/step - loss: 0.1126 - accuracy: 0.9882 - val_loss: 0.3615 - val_accuracy: 0.8438
Epoch 7/80
45/45 [==============================] - 0s 9ms/step - loss: 0.0822 - accuracy: 0.9951 - val_loss: 0.3628 - val_accuracy: 0.8375
Epoch 8/80
45/4

##Using 10000 features

In [12]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

texts = df['text'].tolist()
labels = df['label']

# Initialize TfidfVectorizer for TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_features=10000, tokenizer=lambda x: x.split())
X_tfidf = vectorizer.fit_transform(texts)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Convert sparse matrix to dense array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
print(X_train.shape)

(1600, 10000)


###Logistic Regression

In [14]:
clf = LogisticRegression()

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8425
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       199
           1       0.85      0.83      0.84       201

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



###SVM

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


# Initialize SVM classifier
clf = SVC(kernel='linear', random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.86
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       199
           1       0.87      0.85      0.86       201

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400



###Neural Network

In [16]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(X_train_dense.shape[1],), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_dense, y_train, epochs=80, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred_proba = model.predict(X_test_dense)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Display classification report
print('\nClassification Report:')
print(classification_report(y_test, y_pred))


Epoch 1/80
45/45 [==============================] - 2s 33ms/step - loss: 0.6715 - accuracy: 0.7042 - val_loss: 0.6357 - val_accuracy: 0.8125
Epoch 2/80
45/45 [==============================] - 1s 25ms/step - loss: 0.5181 - accuracy: 0.9097 - val_loss: 0.5290 - val_accuracy: 0.7937
Epoch 3/80
45/45 [==============================] - 1s 25ms/step - loss: 0.3319 - accuracy: 0.9507 - val_loss: 0.4405 - val_accuracy: 0.8188
Epoch 4/80
45/45 [==============================] - 1s 30ms/step - loss: 0.1931 - accuracy: 0.9819 - val_loss: 0.3967 - val_accuracy: 0.8250
Epoch 5/80
45/45 [==============================] - 1s 31ms/step - loss: 0.1200 - accuracy: 0.9917 - val_loss: 0.3810 - val_accuracy: 0.8250
Epoch 6/80
45/45 [==============================] - 2s 41ms/step - loss: 0.0743 - accuracy: 0.9986 - val_loss: 0.3776 - val_accuracy: 0.8313
Epoch 7/80
45/45 [==============================] - 2s 39ms/step - loss: 0.0501 - accuracy: 0.9993 - val_loss: 0.3773 - val_accuracy: 0.8375
Epoch 8/80
45